In [1]:
# import de libs necessárias para o processo de data cleaning

import numpy as np
import pandas as pd
import zipfile
import os
from datetime import datetime
import json
import psycopg2
import sqlalchemy
from psycopg2 import sql
from sqlalchemy import create_engine

In [ ]:
# diretório onde o dataset sera salvo
dataset_dir = "datasets"
os.makedirs(dataset_dir, exist_ok=True)

# link do dataset no kaggle
kaggle_dataset = "yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring"

# baixando o dataset
os.system(f"kaggle datasets download -d {kaggle_dataset} -p {dataset_dir}")

print(f"Dataset baixado e salvo em: {dataset_dir}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring
License(s): CC0-1.0


In [2]:
# caminho do arquivo ZIP baixado
zip_file_path = os.path.join("datasets", "fifa-21-messy-raw-dataset-for-cleaning-exploring.zip")

# extraindo conteúdo
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/Users/gms4/Desktop/guilherme/TCC/Project")


In [3]:
# transformando csv em df e dando uma olhada nele
df = pd.read_csv("fifa21 raw data v2.csv")
df.head()

/var/folders/md/55dn7z6j7cv20tmwqbwsfbjc0000gq/T/ipykernel_45848/3679858239.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fifa21 raw data v2.csv")


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


## Carregamento: Injetando Dataset do Banco de Dados PostgresQL

In [4]:
# carregar o arquivo JSON com as configurações de conexão
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# conectando no banco postgreSQL
connection = psycopg2.connect(
    host=config['host'],
    port=config['port'],
    database=config['database'],
    user=config['user'],
    password=config['password']
)

In [5]:
# Conectar o psycopg2 com o SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}', creator=lambda: connection)

# Inserir o DataFrame no banco de dados
df.to_sql('fifa_model_elt', engine, if_exists='replace', index=False)

print("DataFrame injetado com sucesso!")

DataFrame injetado com sucesso!
